# Parameterbestimmung bei dichtebasierten Verfahren
Während der einzige Parameter für k-Means mit Hilfe des Silhouettenkoeffizienten relativ einfach gefunden werden kann, sind für DBScan zwei verschiedene Parameter notwendig. In diesem Abschnitt wird daher eine Herangehensweise für die Bestimmung von $min\_pts$ und $\epsilon$ betrachtet.

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

from tui_dsmt.clustering.datasets import clustering_example1

## Inhaltsverzeichnis
- [Datensatz](#Datensatz)
- [Literaturbezug](#Literaturbezug)
- [min_pts](#minpts)
- [$\epsilon$](#epsilon)
- [Ergebnis](#Ergebnis)
- [Zusammenfassung](#Zusammenfassung)

## Datensatz
Wir verwenden den bereits häufiger in Erscheinung getretenen Datensatz mit $15$ Häufungen in zwei Dimensionen.

In [ ]:
px.scatter(clustering_example1, x='x', y='y', color='c')

## Literaturbezug
DBScan wurde im Paper mit dem Titel [*A Density-Based Algorithm for Discovering Clusters in Large Spatial Databases with Noise*](https://cdn.aaai.org/KDD/1996/KDD96-037.pdf) von Ester et al. (1996) vorgestellt.

Sander et al. (1998) haben in ihrem Paper [*Density-Based Clustering in Spatial Databases: The Algorithm GDBSCAN and Its Applications*](https://link.springer.com/article/10.1023/A:1009745219419) den Algorithmus GDBSCAN - eine Generalisierung von DBScan - vorgestellt. Sie gehen dabei auch auf die Parameterbestimmung für DBScan ein.

## min_pts
Für zweidimensionale Daten empfehlen Ester et al. $k = 4$ zu wählen und $min\_pts$ ebenfalls auf diesen Wert zu setzen.

In [ ]:
k = 4
min_pts = 4

Sander et al. empfehlen im Allgemeinen ein $k$ zu wählen, das in etwa $2 * dim - 1$ entspricht, wobei $dim$ die Dimension der Daten beschreibt. Kleine Änderungen für $k$ führten in ihren Experimenten aber nicht zu essentiellen Änderungen im Ergebnis. $min\_pts$ ergibt sich dann als $k + 1$ bzw $2 * dim$. Für zweidimensionale Daten wird also geraten, $k = 3$ und $min\_pts = 4$ zu wählen.

## $\epsilon$
Für den Parameter $\epsilon$ lässt sich dagegen keine einfache Empfehlung formulieren, die ausschließlich auf der Anzahl der Dimensionen basiert. Stattdessen soll $\epsilon$ so gewählt werden, dass der Algorithmus Rauschen von tatsächlichen Clusterpunkten trennt. Die Wahl ist also abhängig vom Datensatz.

Zusammen mit $min\_pts$ wurde auch ein $k$ bestimmt. Dieses wird nun verwendet, um ein sogenanntes $k$-Distanz Diagramm zu erzeugen. Die $k$-Distanz ist dabei der durchschnittliche Abstand zu den $k$ nächsten Nachbarn eines spezifischen Objekts im Datensatz und wird auf der y-Achse abgetragen. Auf der x-Achse sind alle Objekte im Datensatz abgetragen, nachdem sie absteigend nach der ihnen zugeordneten $k$-Distanz sortiert wurden. (Der Aufwand bei der Berechnung der $k$-Distanz ist ein gutes Argument, $k$ eher klein zu wählen.)

Für den Datensatz sieht das $(k\!=\!4)$-Distanz Diagramm wie folgt aus.

In [ ]:
neighbors = NearestNeighbors(n_neighbors=k)
neighbors_fit = neighbors.fit(clustering_example1[['x', 'y']])

distances, _ = neighbors_fit.kneighbors(clustering_example1[['x', 'y']])
sorted_distances = np.sort(distances, axis=0)[:,1]

fig = px.line(
    pd.DataFrame({ 'o': range(len(sorted_distances), 0, -1), 'd': sorted_distances }),
    x='o', y='d'
)
fig.add_vline(x=26, line_width=0.5, line_dash='dash', line_color='red')

fig

Das Diagramm kann nun interpretiert werden: Der Benutzer sucht *visuell* ein Grenzobjekt, das sich im ersten *Tal* befindet. Dieses Tal stellt eine Art Knick in der Darstellung dar, an dem sich die Verringerung der $k$-Distanz zwischen den Objekten das erste Mal deutlich verlangsamt. Links dieses Grenzobjekts befindet sich das Rauschen - diese Objekte haben eine relativ hohe $k$-Distanz zu ihren Nachbarn und sollten demnach keinen Clustern zugeordnet werden - und rechtsseitig die Clusterpunkte - diese Objekte haben eine im Vergleich geringe $k$-Distanz zu ihren Nachbarn und sollten Clustern zugeordnet werden.

$\epsilon$ wird nun als $k$-Distanz des Grenzobjekts gewählt. Sie können die $k$-Distanz entweder auf der $y$-Achse ablesen oder alternativ die Interaktivität von Plotly verwenden, um den Wert für ein mit der Maus markiertes Objekt zu erhalten.

Im Diagramm ist **beispielhaft** das Objekt mit der Nummer $26$ und einer $k$-Distanz von $0.2808$ markiert. Wählen Sie dagegen ein Grenzobjekt, das sich weiter links befindet, vergrößert sich $\epsilon$ und weniger Punkte werden als Rauschen erkannt. Wählen Sie ein Grenzobjekt, das sich weiter rechts befindet, wird $\epsilon$ geringer und die Rauscherkennung im Algorithmus aggressiver.

In [ ]:
e = 0.2808

## Ergebnis
Die nachfolgende Zelle erzeugt ein Clustering mit DBScan mit Hilfe der ermittelten Parameter. In der Legende können Sie das Rauschen ($-1$) ein- und ausblenden.

In [ ]:
clustering_example1['dbscan'] = DBSCAN(eps=e, min_samples=min_pts).fit_predict(clustering_example1[['x', 'y']]).astype(str)
px.scatter(clustering_example1, x='x', y='y', color='dbscan')

Probieren Sie auch andere Grenzobjekte aus.

## Zusammenfassung
In den wissenschaftlichen Arbeiten zu DBScan und davon abgeleiteten Algorithmen lassen sich Hinweise zur korrekten Parameterwahl finden. Mit Python und den bekannten Bibliotheken umgesetzt, erlauben Sie ein einfaches Auffinden von $min\_pts$ und $\epsilon$.